### **1. Librerias**

In [95]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access variables
email = os.getenv('email')
password = os.getenv('password')

print(f"Email: {email}")

Email: elianapinto20@outlook.com


In [96]:
# librerias 
import pandas as pd
import sys
import time


In [97]:
import warnings

# Ignorar warnings de pandas sobre cambios futuros
warnings.filterwarnings('ignore', category=FutureWarning)
import logging


In [98]:
# Librerias Selenium
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By  # buscar por etiquetas, clases, ids, paths, xpaths
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [99]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

### **2. Webscrapping data**

In [100]:
def pagina_principal(url):
    # agregar opciones al driver
    options = Options()

    # Ajustar el custom User-Agent, evitar bloqueos facilmente
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")

    # Inicializar el webdriver con las opciones personalizadas. 
    driver = webdriver.Chrome(options=options)

    # maximizar pantalla
    driver.maximize_window()

    # Entra a la pagina
    driver.get(url)
    return driver

In [101]:
def sign_in(driver):


    try:
        # Espera explícita para el enlace de inicio de sesión
        sign_in_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="signIn"]/div/div/a'))
        )
        sign_in_link.click()

        # Espera explícita para el botón de inicio de sesión
        sign_in_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.gr-button.gr-button--dark.gr-button--auth.authPortalConnectButton.authPortalSignInButton'))
        )
        sign_in_button.click()

        # Encuentra el campo de correo electrónico y envía el correo
        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_email"))
        )
        email_input.send_keys(f"{email}")

        # Encuentra el campo de contraseña y envía la contraseña
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_password"))
        )
        password_input.send_keys(f"{password}")

        # Encuentra y haz clic en el botón de envío
        sign_in_submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "signInSubmit"))
        )
        sign_in_submit_button.click()

    except Exception as e:
        print(f"Se produjo un error: {e}")


In [102]:
### NEW ! 
# def iniciar_y_logear(driver, url):
#     # Llamamos a la función para abrir la página
#     driver = iniciar_driver(url)
    
#     # Llamamos a la función para iniciar sesión después de abrir la página
#     sign_in(driver)

# # Llamamos a la función que incluye ambas acciones
# iniciar_y_logear(driver, 'http://example.com')


In [103]:
def go_to_all_genres(driver):
    
    # Espera hasta que el span con el texto "Browse ▾" sea clickeable
    browse_span = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Browse ▾']"))
    )

    # Haz clic en el span
    browse_span.click()

    # Espera hasta que el enlace "All Genres" sea clickeable
    all_genres_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[text()='All Genres']"))
    )

    # Haz clic en el enlace "All Genres"
    all_genres_link.click()

In [104]:
def create_dict():
    books_inf = {
    'books': [],
    'writer': [],
    'rating': [],
    'description': [],
    'statistics': [],
    'details': [],
    'urls': [],
    'gen': [],
    'reading': [],
    'will_read': [],
    'isbn': [],
    'ediciones': [],
    'published': [],
    'format': [],
    'asin': [],
    'language': []
    }
    return books_inf


In [105]:
def get_genres(driver):
    try:
        # Encuentra todos los nombres de géneros
        elements = driver.find_elements(By.CSS_SELECTOR, ".bigBoxContent a.gr-hyperlink")
        names = [element.text for element in elements]
    except TimeoutException:
        logging.error("Error general: No se pudo cargar la lista de géneros (Timeout).")
    except NoSuchElementException:
        logging.error("Error: No se encontraron los géneros en la página.")
    except WebDriverException as e:
        logging.error(f"Error del navegador")
    except Exception as e:
        logging.error(f"Error inesperado: {e}")

    return names[5:6]

In [106]:
# //[@class="actionLink" and contains(text(), "More {gen} books...")]
        
        ## AQUI VA UNA FUNCION QUE BUSCA LOS LIBROS Y LUEGO LOS REVISA Y SACA LA INFO 
        ## DEBO VOLVER ATRAS PORQUE TIENE VARIOS LINKS LA PAGINA O? 


        # # Regresar nuevamente a la página anterior
        # driver.back()
        # time.sleep(2)  # Pausa para permitir la carga de la página

In [107]:
def select_a_genre(driver, genres):

    for gen in genres:
        # Este rango parece estar limitado a 1 género, puedes ajustar el rango según sea necesario
        logging.info(f"Procesando género: {gen}")
        
        try:
            # Esperar que el enlace sea clickeable y hacer clic
            art_link = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, f"//a[contains(text(), '{gen}')]"))
            )
            art_link.click()
            time.sleep(1)  # Pausa para permitir la carga de la página
            
            logging.info(f"Navegando a: {driver.current_url}")
            return gen
        

        except TimeoutException:
            logging.error(f"Error: No se pudo hacer clic en el enlace para '{gen}' (Timeout).")
        except NoSuchElementException:
            logging.error(f"Error: No se encontró el enlace para '{gen}'.")
        except WebDriverException as e:
            logging.error(f"Error del navegador: {e}")


In [108]:
# def get_links(driver): 
#     try: 
#         logging.info("Proceeding with scraping book URLs...")
#         # Usamos un conjunto para evitar duplicados
#         URLs = set()
        
#         # Encontramos los elementos <a> que empiezan con '/book/show'
#         anchor_tags = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/book/show')]")
        
#         # Iteramos sobre los elementos encontrados
#         for tag in anchor_tags:
#             url = tag.get_attribute('href')
#             if url in URLs:
#                 logging.info(f"Duplicate URL skipped: {url}")
#             else:
#                 URLs.add(url)
#                 logging.info(f"Found unique book URL: {url}")
        
#         # Convertimos el conjunto en una lista para devolverlo
#         return list(URLs)
    
#     except Exception as e:
#         logging.error("Error occurred while trying to find book URLs", exc_info=True)
#         return []


In [109]:
# def scroll_until_found(driver, xpath):
#     for _ in range(10):  # Intenta 10 desplazamientos como máximo
#         try:
#             button = driver.find_element(By.XPATH, xpath)
#             driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
#             return button
#         except NoSuchElementException:
#             driver.execute_script("window.scrollBy(0, 500);")
#             time.sleep(1)  # Espera para permitir que se cargue más contenido
#     raise Exception(f"No se encontró el elemento con XPath: {xpath}")

# def explore_books(driver, gen): 
#     try:
#         gen_formatted = gen.lower()  # Convierte "art" a "Art"
#         xpath = f'//div[@class="moreLink"]/a[contains(text(), "More {gen_formatted} books...")]'
#         button = scroll_until_found(driver, xpath)
#         button.click()
#         url_books = get_links(driver)
#         print(f"Botón 'More {gen} books...' clickeado exitosamente.")
#     except Exception as e:
#         print(f"No se pudo hacer clic en el botón: {e}")
    
#     return url_books


In [110]:
# logging.info(f"Navegando a: {driver.current_url}")

# # Ahora buscar los enlaces en esta página
# urls = get_links()
# for url in urls:
#     get_information(url, gen)

In [111]:
url = "https://www.goodreads.com" 

In [112]:
driver = pagina_principal(url)

In [113]:
sign_in(driver)

In [114]:
go_to_all_genres(driver)

In [115]:
dict_dataframe = create_dict()

In [116]:
dict_dataframe

{'books': [],
 'writer': [],
 'rating': [],
 'description': [],
 'statistics': [],
 'details': [],
 'urls': [],
 'gen': [],
 'reading': [],
 'will_read': [],
 'isbn': [],
 'ediciones': [],
 'published': [],
 'format': [],
 'asin': [],
 'language': []}

In [117]:
gen = select_a_genre(driver, genres)

In [118]:
gen

'Art'

In [119]:
explore_books(driver, gen)

Botón 'More Art books...' clickeado exitosamente.


['https://www.goodreads.com/book/show/2784.Ways_of_Seeing',
 'https://www.goodreads.com/book/show/2784.Ways_of_Seeing',
 'https://www.goodreads.com/book/show/222078.The_Story_of_Art',
 'https://www.goodreads.com/book/show/222078.The_Story_of_Art',
 'https://www.goodreads.com/book/show/13099738-steal-like-an-artist',
 'https://www.goodreads.com/book/show/13099738-steal-like-an-artist',
 'https://www.goodreads.com/book/show/627206.The_New_Drawing_on_the_Right_Side_of_the_Brain',
 'https://www.goodreads.com/book/show/627206.The_New_Drawing_on_the_Right_Side_of_the_Brain',
 'https://www.goodreads.com/book/show/615570.The_Artist_s_Way',
 'https://www.goodreads.com/book/show/615570.The_Artist_s_Way',
 'https://www.goodreads.com/book/show/187633.Art_and_Fear',
 'https://www.goodreads.com/book/show/187633.Art_and_Fear',
 'https://www.goodreads.com/book/show/114683.Wall_and_Piece',
 'https://www.goodreads.com/book/show/114683.Wall_and_Piece',
 'https://www.goodreads.com/book/show/567616.The_Art

In [125]:
books_links = pd.read_csv('../reviews_goodreads_tarea.csv')['book_id']
urls = list(set(list(books_links)))
urls

[1,
 44531714,
 3,
 2,
 49807365,
 5,
 6,
 32571395,
 30375937,
 53215250,
 199458836,
 30244886,
 61407256,
 25,
 110690331,
 30,
 33,
 42369064,
 11501608,
 54689834,
 426025,
 63930414,
 91717683,
 294963,
 294965,
 131123,
 229432,
 10944570,
 294971,
 294972,
 294973,
 32827,
 688191,
 61210688,
 294977,
 193790015,
 61243455,
 65605,
 294983,
 294984,
 294986,
 294988,
 294989,
 11862093,
 32849,
 213680213,
 8552536,
 60784729,
 220987481,
 60194905,
 60489817,
 28180574,
 41025630,
 7176298,
 21954673,
 124944498,
 7864437,
 43843702,
 18481271,
 17924213,
 60457077,
 122,
 30965883,
 41222268,
 60915837,
 48988286,
 34930815,
 57507968,
 123207806,
 52166786,
 10387581,
 59342980,
 16089223,
 58065031,
 58065033,
 50659466,
 50659467,
 50659468,
 62980235,
 50659469,
 50659471,
 50659472,
 146604171,
 58720403,
 79331476,
 45416596,
 214368406,
 3342492,
 29622429,
 211189918,
 57442466,
 65994917,
 50626726,
 38600870,
 40861864,
 60784812,
 24182961,
 38731956,
 34930873,
 6

In [130]:
def create_link_books(urls):
    urls_books_complete = []
    url_base = 'https://www.goodreads.com/book/show/'
    for i in urls:
        url_complete = f"{url_base}{i}"
        urls_books_complete.append(url_complete)
    return urls_books_complete


urls_to_scrap = create_link_books(urls)



In [131]:
urls_to_scrap

['https://www.goodreads.com/book/show/1',
 'https://www.goodreads.com/book/show/44531714',
 'https://www.goodreads.com/book/show/3',
 'https://www.goodreads.com/book/show/2',
 'https://www.goodreads.com/book/show/49807365',
 'https://www.goodreads.com/book/show/5',
 'https://www.goodreads.com/book/show/6',
 'https://www.goodreads.com/book/show/32571395',
 'https://www.goodreads.com/book/show/30375937',
 'https://www.goodreads.com/book/show/53215250',
 'https://www.goodreads.com/book/show/199458836',
 'https://www.goodreads.com/book/show/30244886',
 'https://www.goodreads.com/book/show/61407256',
 'https://www.goodreads.com/book/show/25',
 'https://www.goodreads.com/book/show/110690331',
 'https://www.goodreads.com/book/show/30',
 'https://www.goodreads.com/book/show/33',
 'https://www.goodreads.com/book/show/42369064',
 'https://www.goodreads.com/book/show/11501608',
 'https://www.goodreads.com/book/show/54689834',
 'https://www.goodreads.com/book/show/426025',
 'https://www.goodreads.

In [120]:



# explore_books(driver, gen)


In [121]:
# //*[@id="bodycontainer"]/div[3]/div[1]/div[2]/div[2]/div[7]/div[2]/div/div[5]/a

In [122]:
ss

NameError: name 'ss' is not defined

In [11]:
# url = 'https://www.goodreads.com/book/show/199793433-ingrained'
# driver.get(url)

In [12]:

# # try:
# #     # Espera a que el elemento esté presente en el DOM
# #     element = WebDriverWait(driver, 10).until(
# #         EC.presence_of_element_located((By.CLASS_NAME, "SocialSignalCard__caption"))
# #     )
# #     # Verifica si el subtítulo está dentro del contenedor
# #     captions = element.find_element(By.CLASS_NAME, "SocialSignalsSection__caption")
# #     texto = captions.text
# #     books_inf['reading'].append(texto if texto else None)
# # except (NoSuchElementException, TimeoutException) as e:
# #     logging.error(f"Error finding or clicking the button: {e}")


# # try:
# #     element = WebDriverWait(driver, 20).until(
# #         EC.presence_of_element_located((By.XPATH, "//div[@data-testid='toReadSignal']"))
# #     )
# #     # Verifica si el subtítulo está dentro del contenedor
# #     caption = element.find_element(By.XPATH, "//div[@data-testid='toReadSignal']")
# #     texto = caption.text
# #     books_inf['will_read'].append(texto if texto else None)

# # except (NoSuchElementException, TimeoutException) as e:
# #     logging.error(f"Error finding or clicking the button: {e}")


# # try:
# #     # Use XPath to find the element by its text
# #     element = WebDriverWait(driver, 20).until(
# #         EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Show all 6 editions')]"))
# #     )
    
# #     # Get the text inside the span element
# #     texto = element.text
# #     print(texto)  # Or append it to your desired list
# #     books_inf['ediciones'].append(texto if texto else None)

# # except (NoSuchElementException, TimeoutException) as e:
# #     logging.error(f"Error finding the text: {e}")

# try:
#     # Find the "Format" text
#     format_text = WebDriverWait(driver, 20).until(
#         EC.presence_of_element_located((By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][1]//div[contains(@class, 'TruncatedContent__text')]"))
#     ).text

#     # Find the "Published" text
#     published_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][2]//div[contains(@class, 'TruncatedContent__text')]").text

#     # Find the "ISBN" text
#     isbn_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][3]//div[contains(@class, 'TruncatedContent__text')]").text

#     # Find the "ASIN" text
#     asin_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][4]//div[contains(@class, 'TruncatedContent__text')]").text

#     # Find the "Language" text
#     language_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][5]//div[contains(@class, 'TruncatedContent__text')]").text

#     # Print the extracted texts (or store them in a dictionary)
#     print("Format:", format_text)
#     print("Published:", published_text)
#     print("ISBN:", isbn_text)
#     print("ASIN:", asin_text)
#     print("Language:", language_text)

#     # Optionally, append them to your dictionary or list
#     books_inf['format'].append(format_text if format_text else None)
#     books_inf['published'].append(published_text if published_text else None)
#     books_inf['isbn'].append(isbn_text if isbn_text else None)
#     books_inf['asin'].append(asin_text if asin_text else None)
#     books_inf['language'].append(language_text if language_text else None)

# except (NoSuchElementException, TimeoutException) as e:
#     logging.error(f"Error extracting data: {e}")
   

In [13]:
# books_inf

In [14]:
def get_information(url, gen):

    books_inf['gen'].append(gen)
    books_inf['urls'].append(url)

    try:
        logging.info(f"Processing URL: {url}")  # Log the URL being processed

        if url:
            # Navigate to the book's page
            driver.get(url)
            # Extraer nombre del libro
            try:
                names_book = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.Text.Text__title1')))
                name = names_book.text.strip()
                books_inf['books'].append(name if name else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting book name from {url}: {e}")

            # Extraer autores
            try:
                autores = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.ContributorLink__name')))
                name_writer = [autor.text.strip() for autor in autores if autor.text.strip()]
                books_inf['writer'].append(name_writer if name_writer else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting authors from {url}: {e}")

            # Extraer rating
            try:
                ratings_num = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.RatingStatistics__rating')))
                ratings_num_text = ratings_num.text.strip()
                books_inf['rating'].append(ratings_num_text if ratings_num_text else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting rating from {url}: {e}")

            # Extraer descripción
            try:
                desc = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.Formatted')))
                description = desc.text.strip()
                books_inf['description'].append(description if description else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting description from {url}: {e}")

            # Extraer estadísticas
            try:
                statics = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'RatingStatistics__meta')))
                ratings_book_views = statics[0].get_attribute('aria-label').strip() if statics else None
                books_inf['statistics'].append(ratings_book_views if ratings_book_views else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting statistics from {url}: {e}")

            # Extraer detalles adicionales
            try:
                detail = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.FeaturedDetails p')))
                dets = [det.text.strip() for det in detail if det.text.strip()]
                books_inf['details'].append(dets if dets else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting details from {url}: {e}")


            # Initialize WebDriverWait with a timeout of 20 seconds
            wait = WebDriverWait(driver, 20)

            # Esperar explícitamente a que el botón sea visible y luego hacer clic
            try:
                # Esperar explícitamente a que el botón sea clickeable y luego hacer clic
                button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button--inline') and contains(@aria-label, 'Book details and editions')]"))
                )
                button.click()
                logging.info("Botón de detalles encontrado y clickeado.")

            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error finding or clicking the button: {e}")
                return

            try:
                # Espera a que el elemento esté presente en el DOM
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "SocialSignalCard__caption"))
                )
                # Verifica si el subtítulo está dentro del contenedor
                captions = element.find_element(By.CLASS_NAME, "SocialSignalsSection__caption")
                texto = captions.text
                books_inf['reading'].append(texto if texto else None)
            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error finding or clicking the button: {e}")


            try:
                element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@data-testid='toReadSignal']"))
                )
                # Verifica si el subtítulo está dentro del contenedor
                caption = element.find_element(By.XPATH, "//div[@data-testid='toReadSignal']")
                texto = caption.text
                books_inf['will_read'].append(texto if texto else None)

            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error finding or clicking the button: {e}")


            # try:
            #     # Buscar el elemento que contiene "Show all" independientemente del número de ediciones
            #     element = WebDriverWait(driver, 20).until(
            #         EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Show all')]"))
            #     )
                
            #     # Obtener el texto del elemento encontrado
            #     texto = element.text
            #     if texto:
            #         print(f"Texto encontrado: {texto}")  # Mostrar el texto en la consola
            #         books_inf['ediciones'].append(texto)  # Añadir el texto a la lista
            #     else:
            #         logging.info("El elemento está presente pero no contiene texto.")
            #         books_inf['ediciones'].append(None)

            # except TimeoutException:
            #     logging.warning("El elemento 'Show all' no está presente en esta URL.")
            #     books_inf['ediciones'].append(None)  # Registrar que no había texto

            # except NoSuchElementException as e:
            #     logging.error(f"Error al buscar el texto: {e}")
            #     books_inf['ediciones'].append(None)  # Registrar que no había texto

            try:
                # Find the "Format" text
                format_text = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][1]//div[contains(@class, 'TruncatedContent__text')]"))
                ).text

                # Find the "Published" text
                published_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][2]//div[contains(@class, 'TruncatedContent__text')]").text

                # Find the "ISBN" text
                isbn_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][3]//div[contains(@class, 'TruncatedContent__text')]").text

                # Find the "ASIN" text
                asin_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][4]//div[contains(@class, 'TruncatedContent__text')]").text

                # Find the "Language" text
                language_text = driver.find_element(By.XPATH, "//dl[@class='DescList']//div[contains(@class, 'DescListItem')][5]//div[contains(@class, 'TruncatedContent__text')]").text

                # Optionally, append them to your dictionary or list
                books_inf['format'].append(format_text if format_text else None)
                books_inf['published'].append(published_text if published_text else None)
                books_inf['isbn'].append(isbn_text if isbn_text else None)
                books_inf['asin'].append(asin_text if asin_text else None)
                books_inf['language'].append(language_text if language_text else None)

            except (NoSuchElementException, TimeoutException) as e:
                logging.error(f"Error extracting data: {e}")

    except Exception as e:
        logging.error(f"An unexpected error occurred for URL {url}: {e}")



In [15]:
# gen= 'Art'

In [16]:
# url = 'https://www.goodreads.com/book/show/199793433-ingrained'
# get_information(url, 'art')

In [ ]:
# Función principal para navegar por las categorías y páginas
def main():
    try:
        # Encuentra todos los nombres de géneros
        elements = driver.find_elements(By.CSS_SELECTOR, ".bigBoxContent a.gr-hyperlink")
        names = [element.text for element in elements]
        
        # Selecciona un rango de géneros para iterar
        for gen in names[5:12]:  # Este rango parece estar limitado a 1 género, puedes ajustar el rango según sea necesario
            logging.info(f"Procesando género: {gen}")
            
            try:
                # Esperar que el enlace sea clickeable y hacer clic
                art_link = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//a[contains(text(), '{gen}')]"))
                )
                art_link.click()
                time.sleep(1)  # Pausa para permitir la carga de la página
                
                logging.info(f"Navegando a: {driver.current_url}")
                
                # Ahora buscar los enlaces en esta página
                urls = get_links()
                for url in urls:
                    get_information(url, gen)

            except TimeoutException:
                logging.error(f"Error: No se pudo hacer clic en el enlace para '{gen}' (Timeout).")
            except NoSuchElementException:
                logging.error(f"Error: No se encontró el enlace para '{gen}'.")
            except WebDriverException as e:
                logging.error(f"Error del navegador: {e}")

            
            # logging.info(f"Regresaste a: {driver.current_url}")
            
            # Intentar hacer clic en el enlace "More art books..."
            try:
                art_link = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, f'//a[@class="actionLink" and @href="/shelf/show/{gen}"]'))
                )
                driver.execute_script("arguments[0].scrollIntoView();", art_link)
                driver.execute_script("arguments[0].click();")
                
                logging.info(f"Se hizo clic en 'More art books...' para {gen}.")
                
                # Ahora buscar enlaces en esta página también
                urls = get_links()
                for url in urls:
                    get_information(url, gen)

                # # Regresar nuevamente a la página anterior
                # driver.back()
                # time.sleep(2)  # Pausa para permitir la carga de la página

            except Exception as e:
                logging.error(f"Error al intentar hacer clic en 'More art books...' para {gen}: {e}")


            # Regresar a la página anterior
            # url = '' 
            # driver.()
            # time.sleep(2)  # Pausa para permitir la carga de la página
            
    except TimeoutException:
        logging.error("Error general: No se pudo cargar la lista de géneros (Timeout).")
    except NoSuchElementException:
        logging.error("Error: No se encontraron los géneros en la página.")
    except WebDriverException as e:
        logging.error(f"Error del navegador: {e}")
    except Exception as e:
        logging.error(f"Error inesperado: {e}")

# Llamar a la función principal
if __name__ == "__main__":
    main()
    # driver.quit()  # Cerrar el navegador cuando termine el script

In [ ]:
books_inf.keys()

In [ ]:
len(books_inf['description'])


In [ ]:
books_inf

In [ ]:
ss

In [13]:
# try:
#     # Encuentra todos los nombres de los géneros
#     elements = driver.find_elements(By.CSS_SELECTOR, ".bigBoxContent a.gr-hyperlink")
#     names = [element.text for element in elements]
    
#     for gen in names[5:6]:
#         print(gen)
        
#         try:
#             print(f"Procesando: {gen}")
            
#             # Esperar que el enlace sea clickeable y hacer clic
#             art_link = WebDriverWait(driver, 10).until(
#                 EC.element_to_be_clickable((By.XPATH, f"//a[contains(text(), '{gen}')]"))
#             )
#             time.sleep(1)  # Pausa manual antes de hacer clic (opcional)
#             art_link.click()
            
#             # Opcional: Realizar alguna acción en la nueva página
#             print(f"Navegando a: {driver.current_url}")
#             time.sleep(2)  
            
#             # Configure logging
#             logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

#             try:
#                 # Try to find and click on the "Art" link
#                 art_news = WebDriverWait(driver, 10).until(
#                     EC.element_to_be_clickable((By.XPATH, '//*[@id="bodycontainer"]/div[3]/div[1]/div[2]/div[2]/div[4]/div[2]/div/div[5]/a'))
#                 )
#                 art_news.click()

#                 logging.info("No botton to click.")
#                 urls = get_links()
#                 for url in urls: 
#                     get_information(url)

#             except Exception as e:
#                 # Log error if "Art" link is not clickable or doesn't exist
#                 logging.error(f"Error occurred while trying to click on 'Art_news' link")
                
#                 urls = get_links()
#                 for url in urls: 
#                     get_information(url)

#                         # Go back to the previous page
#             driver.back()
#             time.sleep(2)  # Adding a small delay to prevent rapid page reloads
#             logging.info(f"Current URL after going back: {driver.current_url}")
#                 # details.append([None])
#             try:
#                 # Wait for the "More art books..." link to be visible and clickable
#                 art_link = WebDriverWait(driver, 10).until(
#                     EC.visibility_of_element_located((By.XPATH, f'//a[@class="actionLink" and @href="/shelf/show/{gen}"]'))
#                 )
#                 # Scroll the element into view (if necessary)
#                 driver.execute_script("arguments[0].scrollIntoView();", art_link)
#                 # Click the link using JavaScript (in case regular click doesn't work)
#                 driver.execute_script("arguments[0].click();", art_link)

#                 logging.info("Successfully clicked on 'More art books...' link.")
#                 # puede que falte darle scrioll tiene 100 paginas/ se puede solo esta!
#                 urls = get_links()
#                 for url in urls: 
#                     get_information(url)

#             except Exception as e:
#                 logging.error(f"Error occurred while trying to click on 'More art books...' link: {e}")
            
#             # Regresar a la página anterior
#             driver.back()
#             time.sleep(2)  # Pausa para garantizar que la página anterior esté cargada
#             # Verificar que regresaste a la página correcta (opcional)
#             print(f"Regresaste a: {driver.current_url}")

#         except TimeoutException:
#             print(f"Error: No se pudo hacer clic en el enlace para '{gen}' (Timeout).")
#         except NoSuchElementException:
#             print(f"Error: No se encontró el elemento para '{gen}'.")
#         except WebDriverException as e:
#             print(f"Error del navegador: {e}")
            
# except TimeoutError:
#     print(f"Ocurrió un error general: {gen}")

# # finally:
# #     # Cerrar el navegador
# #     driver.quit()

In [ ]:
books_inf


In [ ]:
ss

In [11]:
# def retry_get_url(url, retries=2):

#     attempt = 0
#     while attempt < retries:
#         try:
#             driver.get(url)
#             return True
#         except Exception as e:
#             print(f"Error accessing {url}, attempt {attempt + 1}: {e}")
#             time.sleep(3)
#             attempt += 1
#     return False

In [ ]:
URLS

In [ ]:

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

try:
    # Try to find and click on the "Art" link
    art_news = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="bodycontainer"]/div[3]/div[1]/div[2]/div[2]/div[4]/div[2]/div/div[5]/a'))
    )
    art_news.click()
    logging.info("No botton to click.")
except Exception as e:
    # Log error if "Art" link is not clickable or doesn't exist
    logging.error(f"Error occurred while trying to click on 'Art_news' link")
    get_links()
    get_information(URLS)

try:
    # Wait for the "More art books..." link to be visible and clickable
    art_link = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//a[@class="actionLink" and @href="/shelf/show/art"]'))
    )
    # Scroll the element into view (if necessary)
    driver.execute_script("arguments[0].scrollIntoView();", art_link)
    # Click the link using JavaScript (in case regular click doesn't work)
    driver.execute_script("arguments[0].click();", art_link)
    logging.info("Successfully clicked on 'More art books...' link.")

    # puede que falte darle scrioll tiene 100 paginas/ se puede solo esta!
    get_links()
    get_information(URLS)

except Exception as e:
    logging.error(f"Error occurred while trying to click on 'More art books...' link: {e}")




In [ ]:
book_inf

In [ ]:
ss

In [ ]:
# Go back to the previous page
driver.back()

# Optionally, you can check the current URL to verify you're back
print(driver.current_url)

In [ ]:

#     # If the "Art" link isn't found, proceed to scrape URLs from books
#     try:
#         logging.info("Proceeding with scraping book URLs...")
#         anchor_tag = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/book/show')]")
        
#         # Iterate over anchor tags and collect URLs
#         for get_href in anchor_tag:
#             url = get_href.get_attribute('href')
#             urls.append(url)
#             logging.info(f"Found book URL: {url}")
        
#             try:
#                 logging.info(f"Processing URL: {url}")  # Log the URL being processed
            
#                 if url:
#                     urls.append(url)

#                     # Navigate to the book's page
#                     driver.get(url)
#                     button = driver.find_element(By.XPATH, "//button[@aria-label='Book details and editions']")
#                     button.click()

#                     # Try to extract book name
#                     try:
#                         names_book = driver.find_element(By.CSS_SELECTOR, 'h1.Text.Text__title1')
#                         name = names_book.text.strip()
#                         books.append(name if name else None)
#                     except (NoSuchElementException, TimeoutException) as e:
#                         logging.error(f"Error extracting book name from {url}: {e}")
#                         books.append(None)

#                     # Try to extract authors
#                     try:
#                         autores = driver.find_elements(By.CSS_SELECTOR, 'span.ContributorLink__name')
#                         name_writer = [autor.text.strip() for autor in autores if autor.text.strip()]
#                         writer.append(name_writer if name_writer else None)
#                     except (NoSuchElementException, TimeoutException) as e:
#                         logging.error(f"Error extracting authors from {url}: {e}")
#                         writer.append(None)

#                     # Try to extract book rating
#                     try:
#                         ratings_num = driver.find_element(By.CSS_SELECTOR, 'div.RatingStatistics__rating').text.strip()
#                         rating.append(ratings_num if ratings_num else None)
#                     except (NoSuchElementException, TimeoutException) as e:
#                         logging.error(f"Error extracting rating from {url}: {e}")
#                         rating.append(None)

#                     # Try to extract book description
#                     try:
#                         desc = driver.find_element(By.CSS_SELECTOR, 'span.Formatted').text.strip()
#                         description.append(desc if desc else None)
#                     except (NoSuchElementException, TimeoutException) as e:
#                         logging.error(f"Error extracting description from {url}: {e}")
#                         description.append(None)

#                     # Try to extract views and reviews statistics
#                     try:
#                         statics = driver.find_elements(By.CLASS_NAME, 'RatingStatistics__meta')
#                         ratings_book_views = statics[0].get_attribute('aria-label').strip() if statics else None
#                         statistics.append(ratings_book_views if ratings_book_views else None)
#                     except (NoSuchElementException, TimeoutException) as e:
#                         logging.error(f"Error extracting statistics from {url}: {e}")
#                         statistics.append(None)

#                     # Try to extract detailed information
#                     try:
#                         detail = driver.find_elements(By.CSS_SELECTOR, 'div.FeaturedDetails p')
#                         dets = [det.text.strip() for det in detail if det.text.strip()]
#                         details.append(dets if dets else [None])
#                     except (NoSuchElementException, TimeoutException) as e:
#                         logging.error(f"Error extracting details from {url}: {e}")
#                         details.append([None])

#             except NoSuchElementException as e:
#                 logging.error(f"Element not found at URL {url}: {e}")
#                 url_missing.append(url)

#             finally:
#                 # Go back to the previous page
#                 driver.back()
#                 time.sleep(2)  # Adding a small delay to prevent rapid page reloads
#                 logging.info(f"Current URL after going back: {driver.current_url}")
        
#     except Exception as e:
#         # Log error if no book URLs are found
#         logging.error(f"Error occurred while trying to find book URLs: {e}")



# # After scraping, log the collected data
# logging.info(f"Books: {books}")
# logging.info(f"Writers: {writer}")
# logging.info(f"Ratings: {rating}")
# logging.info(f"Description: {description}")
# logging.info(f"Statistics: {statistics}")
# logging.info(f"Details: {details}")
# logging.info(f"URLs missing: {url_missing}")



# # Print the collected data (for demonstration)
# print("Books:", books)
# print("Writers:", writer)
# print("Ratings:", rating)
# print("Description:", description)
# print("Statistics:", statistics)
# print("Details:", details)

In [ ]:
ss

In [ ]:
button = driver.find_element(By.XPATH, "//button[@aria-label='Book details and editions']")
button.click()

In [ ]:
url[0:1]

In [ ]:
# aqui estan los links de los libros por genero de libro.
element = driver.find_elements(By.CLASS_NAME, 'gr-hyperlink')
element[0:3]

In [ ]:
url_gen = []
num_books = [] # numero libros por categoria

# Define a function to extract URLs from the page
def extract_urls():
    """
    Esta funcion extrae los urls de los generos que los libros que se encuentran por genero de los links de genero.

    """
    try:
        num = 0
        # Find elements with the combined class using XPath
        elements = driver.find_elements(By.CSS_SELECTOR, 'div.leftAlignedImage.bookBox div a')
        
        for element in elements:
            time.sleep(0.01)
            url = element.get_attribute('href')
            if url not in url_gen:
                url_gen.append(url)
                num += 1 
        num_books.append(num)

    except NoSuchElementException:
        print("No link found on the page.")

    

for i in urls:
    if i:  # Ensure the URL is valid before trying to process it
        driver.get(i)
        extract_urls()

print(url_gen)  # Output the list of generated URLs


In [ ]:
# aqui estan los links de los libros por genero de libro.
element = driver.find_elements(By.CLASS_NAME, 'gr-hyperlink')
element[0:3]

**Procedimiento.**
1. Primero se tomaran los links de los distintos generos de libros que hay en la pagina.
2. Luego se iterara sobre esta lista de url con los generos para tomar los libros pertenecientes a cada clase.
3. Se cuentas cuantos libro aparecen por genero.
4. Con la lista de generos se itera sobre cada url para tomar los urls de los libros para cada genero.
5. Para cada url de cada libro se itera y se extrae la siguiente informacion: 
> - Nombre del libro - titulo
> - Autor o autores
> - Calificacion
> - Estadisticas, reviews y votos
> - Descripcion 
> - Detalles, fecha e informacion del libro

### **Link por genero**

In [ ]:
# Recuperar links de generos. 
urls = []

for item in element:
    try:
        # URL
        url = item.get_attribute('href')
        urls.append(url)
      
    except:
        NoSuchElementException

urls = urls
urls

In [13]:
# urls por genero
urls = urls[3:32]

Se guardan los links en un dataframe para pasarlos a un csv asi no estar recogiendo esta informacion cada vez que se ejecuta el programa.

In [ ]:
# Guarda en un csv.
dt = pd.DataFrame({"url_gen": urls,
                  "books_amount": num_books})
dt.to_csv('link_books.csv', index=False)

In [46]:
# leer los los datos de links por genero.
dt = pd.read_csv('link_genre.csv')

In [ ]:
dt.head(29)

In [ ]:
dt.shape

In [14]:
urls = dt['url_gen'] 

In [ ]:
urls

### **Link para cada libro por genero**

In [ ]:
url_gen = []
num_books = [] # numero libros por categoria

# Define a function to extract URLs from the page
def extract_urls():
    """
    Esta funcion extrae los urls de los generos que los libros que se encuentran por genero de los links de genero.

    """
    try:
        num = 0
        # Find elements with the combined class using XPath
        elements = driver.find_elements(By.CSS_SELECTOR, 'div.leftAlignedImage.bookBox div a')
        
        for element in elements:
            time.sleep(0.01)
            url = element.get_attribute('href')
            if url not in url_gen:
                url_gen.append(url)
                num += 1 
        num_books.append(num)

    except NoSuchElementException:
        print("No link found on the page.")

    

for i in urls:
    if i:  # Ensure the URL is valid before trying to process it
        driver.get(i)
        extract_urls()

print(url_gen)  # Output the list of generated URLs



Igualmente se guardan los links para cada libro por genero en un csv.

In [ ]:
dt = pd.DataFrame({"url_gen": url_gen})
dt.to_csv('link_books_all.csv', index=False)

In [8]:
dt = pd.read_csv('link_books_all.csv')
url_gen = dt.url_gen

In [ ]:
# 837 urls con libros
len(url_gen)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

# Assuming driver and url_gen are already defined
url_missing = []
fechas = []
reviews = []

for url in url_gen[0:2]:  # Loops through all URLs
    print(f"Processing URL: {url}")
    if url:
        try:
            driver.get(url)
            
            # Wait for the element containing the reviewer's name to be visible
            wait = WebDriverWait(driver, 10)  # 10 seconds wait for elements
            fecha_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.ReviewerProfile__name')))
            
            # Extract the text from each element
            for fecha in fecha_elements:
                fechas.append(fecha.text)
            
            print(f"Extracted dates/review info: {fechas[-1]}")  # Print last extracted item

            # Wait for the element containing the reviewer's name to be visible
            wait = WebDriverWait(driver, 10)  # 10 seconds wait for elements
            reviewss = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div span.Formatted')))
            
            # Extract the text from each element
            for fecha in reviewss:
                reviews.append(fecha.text)

        except NoSuchElementException:
            print(f"Elemento no encontrado en la URL: {url}")
            url_missing.append(url)
        except Exception as e:
            print(f"Error occurred for URL {url}: {e}")
            url_missing.append(url)

# Print out the missing URLs
print("URLs with missing elements:", url_missing)
print("Extracted dates:", fechas)



In [ ]:
len(fechas)

In [ ]:
len(reviews)

In [ ]:
reviews

In [ ]:
ss

### **Extraccion de la Informacion de los libros**

In [ ]:
# listas con la informacion a extraer
url_missing = []
books = []
writer = []
rating = []
statistics = []
description = []
details = []

for url in url_gen[0:10]: 
    print(url)
    if url:
        name_writer = []
        dets = []
        try:
            driver.get(url)
            time.sleep(0.01)

            # driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[7]/div/div/button/span[1]').click()

            # Cod = driver.find_element(By.CSS_SELECTOR, 'div.EditionDetails')
            # cod = cod.text
            # print(cod)

            
            # Nombre libro
            names_book = driver.find_element(By.CSS_SELECTOR, 'h1.Text.Text__title1')
            name = names_book.text
            books.append(name)

            # autores 
            autores = driver.find_elements(By.CSS_SELECTOR, 'span.ContributorLink__name')
            for i in autores:
                autor = i.text
                name_writer.append(autor)
            writer.append(name_writer)

            # Calificacion libro
            ratings_num = driver.find_element(By.CSS_SELECTOR, 'div.RatingStatistics__rating').text
            rating.append(ratings_num)

            #description
            desc = driver.find_element(By.CSS_SELECTOR, 'span.Formatted').text
            description.append(desc)

            # Views y reviews
            statics = driver.find_elements(By.CLASS_NAME, 'RatingStatistics__meta')
            ratings_book_views = statics[0].get_attribute('aria-label')
            statistics.append(ratings_book_views)

            # Extract the details
            detail = driver.find_elements(By.CSS_SELECTOR, 'div.FeaturedDetails p')
            for i in detail:
                det = i.text
                # Verificar si el texto no está vacío antes de agregarlo a la lista
                if det.strip():  # .strip() elimina los espacios en blanco al principio y al final
                    dets.append(det)
            details.append(dets)

        
        except NoSuchElementException:
            print(f"Elemento no encontrado en la URL: {url}")
            url_missing.append(url)


#### **Libros que no se encontro informacion**

Se guardan los links de los libros que no se encuentras estos se pueden buscar especificamente por nombre y tomar la informacion de alli.  
El programa leyó todos los links y obtuvo la informacion de todos los libros.

In [196]:
# Los libros que no se cargan.
url_missing

for i, value in enumerate(url_gen):
    if url_gen[i] == url_missing:
        print(i)

In [ ]:
def name_url_book(url : str) -> str:
    """
    Esta funcion recibe un string en formato url y extrae el nombre del libro y devuelve un string con el nombre del libro.

    Arguments: 
        url: str

    Return 
        str
    
    example:
        >>>url = 'https://www.goodreads.com/book/show/30118.A_Light_in_the_Attic'
        >>>name_url_book(url)
        'A Light in the Attic'
    """

    name = url.split('/')[-1]
    if "-" in name:
        name = " ".join(name.split('-')[1:]).replace("-", " ")
    elif '.' in name:
        name = " ".join(name.split('.')[1:]).replace("_", " ")

    return name

name_url_book(url)

In [20]:
# probar codigo
url_missing = ['https://www.goodreads.com/book/show/30118.A_Light_in_the_Attic']

In [ ]:
# agregar opciones al driver
options = Options()

# Ajustar el custom User-Agent, evitar bloqueos facilmente
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")

# Inicializar el webdriver con las opciones personalizadas. 
driver = webdriver.Chrome(options=options)

# Maximizar la ventana
driver.maximize_window()

# pagina a scrapear
url = "https://www.goodreads.com/search?utf8=%E2%9C%93&query=" 

# Entra a la pagina
driver.get(url)

for i in url_missing:

    # buscar el boton de busqueda
    book = driver.find_element(By.CSS_SELECTOR, "input.searchBox__input.searchBox--large__input")
    book.click()

    # nombre del libro a buscar
    libro = name_url_book(i)
    print(libro)

    # Escribir el nombre del libro en el campo de búsqueda
    book.send_keys(libro, Keys.ENTER)

    time.sleep(5)


    # link = driver.find_element(By.XPATH, "//a[contains(@href, 'book')]")

    # link = driver.find_element(By.XPATH, "//a[contains(@href, 'book')]")
    # link.click


    try:
        # Espera hasta 10 segundos para que el botón de cierre sea visible y luego haz clic en él
        close_ad_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[1]/button/img'))
        )
        close_ad_button.click()
    except:
        print("No se encontró el botón de cierre de la publicidad o no fue clickeable.")


    link = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//a[contains(@href, 'book')]"))
    )


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# agregar opciones al driver
options = Options()

# Ajustar el custom User-Agent, evitar bloqueos facilmente
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")

# Inicializar el webdriver con las opciones personalizadas. 
driver = webdriver.Chrome(options=options)

# Maximizar la ventana
driver.maximize_window()

# pagina a scrapear
url = "https://www.goodreads.com/search?utf8=%E2%9C%93&query=" 

# URL de la página inicial
driver.get(url)

# Iterar sobre la lista de libros
for i in url_missing:
    # Buscar el campo de búsqueda
    book = driver.find_element(By.CSS_SELECTOR, "input.searchBox__input.searchBox--large__input")
    
    # Hacer clic en el campo de búsqueda
    book.click()

    # Obtener el nombre del libro a buscar
    libro = name_url_book(i)
    print(libro)

    # Escribir el nombre del libro en el campo de búsqueda
    book.send_keys(libro, Keys.ENTER)

    # Manejar la publicidad (si aparece)
    try:
        # Esperar hasta que el botón de cierre de la publicidad sea clickeable
        close_ad_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[1]/button/img'))
        )
        close_ad_button.click()
    except:
        print("No se encontró el botón de cierre de la publicidad o no fue clickeable.")
    
    # Esperar a que el enlace del libro sea visible antes de hacer clic
    try:
        link = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//a[contains(@href, '/book/show')]"))
        )
        link.click()  # Hacer clic en el enlace del libro
        # Esperar que la página del libro cargue completamente
        print(f"Accedió a la página del libro: {libro}")
    except:
        print(f"No se encontró el enlace para el libro: {libro}")

    # Pausa para evitar sobrecargar el servidor (puedes ajustarlo o eliminarlo si lo prefieres)
    time.sleep(5)

# Cerrar el navegador después de terminar
driver.quit()


In [ ]:
https://www.goodreads.com/blog/show/2842-readers-most-anticipated-books-of-2025?ref=BigBooks25_eb

In [ ]:
ss

#### **Informacion libros, creacion de dataframe**

In [ ]:
print(books)
len(books)

In [ ]:
print(writer)
len(writer)

In [ ]:
print(rating)
len(rating)

In [ ]:
print(statistics)
len(statistics)

In [ ]:
print(description)
len(description)

In [ ]:
print(details)
len(details)

In [ ]:
len(url_gen[600:])

In [ ]:
data = {'Book_name': books,
        'writers': writer,
        'rating': rating,
        'statistics': statistics,
        'description': description,
        'url': url_gen[600:],
        'details': details}


df = pd.DataFrame(data)
df.to_csv('df_books0_837.csv')

In [ ]:
df.head(5)

Para leer 300 libros el programa toma aproximadamente 25 minutos. Se guardan 3 como archivos separados por coma con 300, 299, y 237 datos. Se utlizaran estos datos para no estar cargando la pagina cada vez que se ejecute el notebook. 

### **Leer Datos, csv**

In [214]:
import pandas as pd

In [216]:
df1 = pd.read_csv('df_books0_300.csv', index_col = 'Unnamed: 0')

In [217]:
df2 = pd.read_csv('df_books0_599.csv', index_col = 'Unnamed: 0')

In [218]:
df3 = pd.read_csv('df_books0_837.csv', index_col = 'Unnamed: 0')

In [ ]:
df1.head(2)

In [ ]:
df2.head(2)

In [ ]:
df3.head(2)

In [313]:
df_total = pd.concat([df1, df2, df3], axis=0)

In [ ]:
df_total.info()

### **Limpieza Datos**


Hay columnas con datos en listas, como es el caso de la columna writers que son los autores del libro. Estos datos se cambian a tipo string para ingresarlos a la base de datos. Hay valores ademas repetidos en estas listas que tambien se remueven.

In [315]:
df_total['writers'] = df_total['writers'].map(lambda x: (", ".join(set(writer.strip() for writer in x.strip().replace("[", "").replace(']', "").split(","))).replace("'",'')))


In [ ]:
df_total.head(6)

La columna statistics tiene dos valores que pueden ser valiosos para el analisis. La cantidad de las personas que califican el libro y la cantidad de reviews que recibe el libro.

In [317]:
for i in df_total["statistics"]:
    ratings, reviews = i.split('and')


df_total['num_ratings'] = df_total['statistics'].map(lambda x: x.split('and')[0].split(" ")[0])


df_total['num_reviews'] = df_total['statistics'].map(lambda x: x.split('and')[1].split(" ")[1])

In [318]:
df_total = df_total.drop(['statistics'], axis = 1)

In [ ]:
df_total.head(2)

In [320]:
df_total['info_book'] = df_total['details'].map(lambda x: x.replace('[','').replace(']','').split(",")[0:2])
df_total["publicacion"] = df_total['details'].map(lambda x: x.replace('[','').replace(']','').split(",")[2:] if len(x.replace('[','').replace(']','').split(",")) > 2 else 'Falta info')
# join(set(writer.strip() for writer in x.strip().replace("[", "").replace(']', "").split(","))).replace("'",'')))

In [321]:
df_total['info_book'] = df_total['info_book'].map(lambda x: (",".join(x)).replace("'",""))
df_total['publicacion'] = df_total['publicacion'].map(lambda x: (",".join(x)).replace("'",""))

In [322]:
df_total = df_total.drop(['details'], axis = 1)

In [ ]:
df_total.head(10)

In [328]:
df_total['description'] = df_total['description'].map(lambda x: x.strip() if pd.notnull(x) else 'nocontent')

In [ ]:
df_total.head(2)

Para los datos que se limpiaron se hace una copia. Luego se procede a guardar los datos en una base de datos.

In [ ]:
df_total.to_csv("final.csv")

In [26]:
df = pd.read_csv('final.csv')

In [ ]:
df

In [ ]:
# Para crear separar los autores que estan en una sola fila. Y asi crear mas tablas. 
#  def add_rows(df, name_column):

#     """ 
#     Esta funcion crea columnas con 2 valores en una columna.
    
#     """
    
#     # Crear un nuevo DataFrame para almacenar las filas separadas
#     new_df = pd.DataFrame(columns=df.columns)

#     # Iterar sobre cada fila del DataFrame original
#     for index, row in df.iterrows():
#         authors = row['writers'].split(', ')  # Separar los autores usando la coma como delimitador
        
#         # Crear una nueva fila para cada autor y agregarla al nuevo DataFrame
#         for author in authors:
#             new_row = row.copy()  # Copiar la fila original
#             new_row['writers'] = author  # Reemplazar la columna 'writers' con un solo autor
#             new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

#     # Mostrar el DataFrame resultante
#     return new_df

# add_rows(df_total, 'writers')

In [ ]:
df_total.isna().sum()

### **Insertar datos en la base de datos**

In [ ]:
# pip install mysql
# pip install mysql-connector-python

In [ ]:
import mysql.connector

In [ ]:
con = mysql.connector.connect(user = "root", password = "123mysequel", host = "127.0.0.1", port =3308)
cursor = con.cursor()

In [ ]:
cursor.execute("CREATE DATABASE BOOKS;")
cursor.execute("USE BOOKS;")

In [ ]:
# SQL query to create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS books_db (
    id_book INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(150),
    rating FLOAT(3,2),
    statistics VARCHAR(100),
    description TEXT,
    year INT,
    information VARCHAR(250),
    num_rating INT,
    num_reviews INT,
    url VARCHAR(200)
);
"""

In [ ]:
# Execute the query
cursor.execute(create_table_query)

In [ ]:
create_table_query = """
ALTER TABLE books_db MODIFY COLUMN year VARCHAR(50);
"""

In [ ]:
# Execute the query
cursor.execute(create_table_query)
con.commit()

In [ ]:
cursor.execute("DESCRIBE books_db;")
result = cursor.fetchall()
for row in result:
    print(row)

In [ ]:
# Commit the changes
con.commit()

In [ ]:
df_total.columns

In [ ]:

# Crear una lista con todos los datos a insertar
data = []
for _, row in df_total.iterrows():
    name = row['Book_name']
    rating = row['rating']
    year = row['year_pub']
    information = row['information']
    num_rating = row['num_ratings']
    num_reviews = row['num_reviews']
    url = row['url']
    description = row['description']

    # Agregar cada fila como una tupla a la lista 'data'
    data.append((name, rating, year, information, num_rating, num_reviews, url, description))

# Ejecutar la inserción masiva utilizando executemany()
add_producto = """
    INSERT INTO books_db 
    (name, rating, year, information, num_rating, num_reviews, url, description) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(add_producto, data)



In [ ]:
# Realizar commit
con.commit()

In [ ]:
cursor.execute("SELECT * FROM books_db;")

rows = cursor.fetchall()

for row in rows:
    print(row)

In [ ]:
con.close()

### **Graficas**


In [ ]:
#pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(x=df_total['rating']);